In [4]:
import numpy as np                       #矩阵操作
import pandas as pd                     #引入
from sklearn import preprocessing        #引入赌热编码，进行数据处理
from pandas import DataFrame
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss     #采用logloss作为评价指标
import matplotlib.pyplot as plt          #画图
import seaborn as sns                    #画图
import time

#import list_utils

%matplotlib inline

from utils import *
from FE_utils import *

### 1、进行数据下采样
先从train数据中按正负样本1：r的比例进行采样，并将其存入本地

In [14]:
#实现正负样本按照1：r的比例采样，并提供存储功能
filename= 'D:/Jupyter/w000/train'
save_name='D:/Down_sample_data.csv'
##filename:原始数据地址+名称，
#save_name：输出文件地址+名称
#r:正负样本比例,默认负样本数是正样本的1
#data_num：所需样本的总量
def down_sample(filename,save_name,r=1,chunksize=100,data_positive_num=10000,save=False,return_=True):
    positive_num=data_positive_num
    negtive_num=data_positive_num*r
    positive_n=0
    negtive_n=0
    datas=pd.read_csv(filename,chunksize=chunksize)
    i=0
#采集正负样本
    for data in datas:
        data_all=pd.read_csv(filename,nrows=1)
        positive=data[data.click==1]
        negtive=data[data.click==0]
#采集正样本        
        if positive_n<positive_num:
            data_all=data_all.append(positive,ignore_index=True)
            positive_n+=positive.shape[0]
#采集负样本           
        if negtive_n<negtive_num:
            if negtive.shape[0]<=positive.shape[0]*r:
                data_all=data_all.append(negtive,ignore_index=True)
                negtive_n+=negtive.shape[0]
            else:
                n=int(positive.shape[0]*r)
                data_all=data_all.append(negtive.head(n),ignore_index=True)
                negtive_n+=n
#存储数据
        if save:
            if i==0:
                data_all.to_csv(save_name,header=True,index_label=False)
            else:
                data_all.to_csv(save_name,mode='a',header=False,index_label=False)            
        i+=1
        if (positive_n>=positive_num)&(negtive_n>=negtive_num):
            break
        
        print('positive_n:'+str(positive_n),'  ','negtive_n:'+str(negtive_n))
        del data
         
        
    if return_:    
        return data_all

#### $$$执行代码1：下采样
#### 参数设置：
filename：数据读取地址；save_name：下采样数据存储地址；r:正负样本比例；chunksize：单次处理数据，建议1w，这样下采样更均匀；data_positive_num：需采正样本数量，建议600w以下

In [15]:
#实现正负样本按照1：r的比例采样，并提供存储功能
filename= 'D:/Jupyter/w000/train'            #数据读取地址
save_name='D:/Down_sample_data.csv'          #下采样数据存储地址
down_sample(filename,save_name,r=1,chunksize=10000,data_positive_num=500000,save=True ,return_=False)
#r:正负样本比例
#chunksize：单次处理数据，建议1w，这样下采样更均匀
#data_positive_num：需采正样本数量，建议600w以下

positive_n:1706    negtive_n:1706
positive_n:3436    negtive_n:3436
positive_n:5276    negtive_n:5276
positive_n:7011    negtive_n:7011
positive_n:8764    negtive_n:8764
positive_n:10470    negtive_n:10470
positive_n:12217    negtive_n:12217
positive_n:13964    negtive_n:13964
positive_n:15692    negtive_n:15692
positive_n:17490    negtive_n:17490
positive_n:19222    negtive_n:19222
positive_n:20972    negtive_n:20972
positive_n:22685    negtive_n:22685
positive_n:24396    negtive_n:24396
positive_n:26117    negtive_n:26117
positive_n:27869    negtive_n:27869
positive_n:29695    negtive_n:29695
positive_n:31396    negtive_n:31396
positive_n:33210    negtive_n:33210
positive_n:34951    negtive_n:34951
positive_n:36600    negtive_n:36600
positive_n:38390    negtive_n:38390
positive_n:40099    negtive_n:40099
positive_n:41825    negtive_n:41825
positive_n:43573    negtive_n:43573
positive_n:45191    negtive_n:45191
positive_n:46706    negtive_n:46706
positive_n:48214    negtive_n:48214
po

### 2、特征工程预处理
#### 2.1、增加用户识别特征 （用于生成新增的7个特征）
通过将device_id+device_model作为用户的唯一识别码，因为联合后在训练、测试数据中的维度较device_id本身仅增加了2.8%。同时删除device_id和device_ip

### $$$执行代码2：  增加用户识别码use_id，删除训练device_id和device_ip
##### 参数设置：
filename：数据读取地址；save_name：数据存储地址；chunksize：每次提取多少数据进行处理，取决于电脑内存

In [17]:
filename= 'D:/Down_sample_data.csv'
save_name='D:/Jupyter/w000/train_23.csv'

data_all=pd.read_csv(filename,chunksize=1000000)

i=0
for data in data_all:
    data['user_id']=data['device_id']+data['device_model']
    data.drop(['device_id','device_ip'],inplace=True,axis=1)
    if i==0:
        data.to_csv(save_name,header=True,index_label=False)
    else  :
        data.to_csv(save_name,mode='a',header=False,index_label=False)
    
    
    del data
    i+=1
    
        
    print('Num_data:'+str(i*2000000))

### 2.2、原19维特征+hour展开3个特征的特征工程
 由于pandas用replace处理大规模数据，非常占内存，故而对获取下采样数据进行分批次读取处理。
  其中hour_days特征的增加用于生成新增的8个特征

#### tip:由于使用特征工程时，replace操作很占内存，故而请使用2.2.2代码分批次读取数据，并将数据存入指定文件

#### 2.2.1、 原始特征工程，生成非onehot编码
      1、实现将除id、click、use_id外取其前1%特征维度的代码
      2、实现将hour生成额外3种特征的代码

In [20]:
#对输入数据进行特征工程
#data：需要进行特征工程的数据；
#name_dir：特征工程所需的dict共两个，19维一个，时间一个
def Feature_Engineering(data_,name_dir='D:/'):
#除hour、id、click、device_id、device_ip外19维特征处理
    start=time.time()
    FE_19_=read_double_dict(name_dir+'FE_19.txt',key_int=False)
#此操作：由txt读取的dict中的数据类型是str，故而需将其还原为int类型的数据
    list_int=['click','hour','C1','banner_pos','device_type','device_conn_type','C14','C15','C16','C17','C18','C19','C20','C21']
    FE_19={}
    for k,v in FE_19_.items():
        dict_={}
        if k in list_int:
            for key,value in v.items():
                dict_[int(key)]=value
        else:
            for key,value in v.items():
                dict_[key]=value        
        FE_19[k]=dict_
    
    for key in FE_19.keys():
        data_[key].replace(FE_19[key],inplace=True)
        
#——————————————————————————————————————————————        
    FE_hour=read_double_dict(name_dir+'FE_hour.txt')
#就hour增加哪个小时、星期几，第几天三类特征    
    data_['hour_hours']=data_.hour.replace(FE_hour['FE_hour_hours'])
    data_['hour_weekday']=data_.hour.replace(FE_hour['FE_hour_weekday'])
    data_['hour_days']=data_.hour.replace(FE_hour['FE_hour_days'])
    
    end=time.time()
    print("times"+str(end-start))
    return data_

#### 2.2.2、 分批次读取要进行特征工程的数据并将其存入一个文件中

In [19]:
##由于采用pandas自带的replace所消耗的内存很大，故而需要分批处理数据并将其存到一个文件中。
#filename：需特征工程的数据
#save_name：文件保存地址
#chunksize：每次对多少数据进行特征工程
#data_num：总共需要多少完成特征工程的数据
filename= 'D:/Jupyter/w000/train'
save_name='D:/FE_data.csv'
def FE_data(filename,save_name,chunksize=100000,data_num=10000000):
    step_num=data_num//chunksize
    print(step_num)
    datas=pd.read_csv(filename,chunksize=chunksize)
    i=0
    for data in datas:
        data_=Feature_Engineering(data)
        if i==0:
            data.to_csv(save_name,header=True,index_label=False)
        else:
            data.to_csv(save_name,mode='a',header=False,index_label=False)
        i+=1
        if i>=step_num:
            break
        del data
    return data_

### $$$执行代码3： 对前99%的数据进行数字编码，剩余1%编码为-1；将时间hour展开为3个新特征
#### 参数设置：
filename：数据读取地址；save_name：数据存储地址，chunksize：每批次处理数据数，建议10w；data_num：总共需处理数据数
预计1h处理300w数据

In [21]:
filename= 'D:/Jupyter/w000/train_23.csv'
save_name='D:/Jupyter/w000/FE_data_train_26.csv'
data_FE=FE_data(filename,save_name,chunksize=100000,data_num=5000000)

50
times124.32160782814026
times124.78383755683899
times125.25509905815125
times123.8774254322052
times125.44318675994873
times126.22883939743042
times127.39102077484131
times125.69707226753235
times126.47747945785522
times127.31162023544312


### 2.3 进行新增的3个关于每小时用户热度的特征工程

##### 1、user_hour_C14：每个用户在小时内查看该广告的次数
##### 2、user_hour_C17：每个用户，在小时内查看该广告告的次数
##### 3、user_day_C14：每个用户，在当天查看该广告的次数

此块代码耗时非常长，原因是先用bygroup生成所需用户热度(频率)的dataframe，再遍历数据根据关键字从dataframe提取对应频率。所处理数据越多
#生成频率的dataframe就越大，在遍历数据查找所消耗的时间就越多。    经测算，600w数据，用单核预计得9天。

#### $$$执行代码4： 生成新的7个表示热度的特征，同时删除'id','hour','user_id'这三个特征
#### $请依次执行以下4块代码
#### 参数设置：
filename：数据读取地址；save_name：生成数据存储地址

In [22]:
filename= 'D:/Jupyter/w000/for_train/FE_train_26.csv'
datas=pd.read_csv(filename,chunksize=1000000)

#### 2.3.1训练集数据按小时拆分

In [2]:
hour_list=[]
for data in range(0,9):
    for hour in range(0,24):
        hour_list.append(14102100+data*100+hour)
hour_list

[14102100,
 14102101,
 14102102,
 14102103,
 14102104,
 14102105,
 14102106,
 14102107,
 14102108,
 14102109,
 14102110,
 14102111,
 14102112,
 14102113,
 14102114,
 14102115,
 14102116,
 14102117,
 14102118,
 14102119,
 14102120,
 14102121,
 14102122,
 14102123,
 14102200,
 14102201,
 14102202,
 14102203,
 14102204,
 14102205,
 14102206,
 14102207,
 14102208,
 14102209,
 14102210,
 14102211,
 14102212,
 14102213,
 14102214,
 14102215,
 14102216,
 14102217,
 14102218,
 14102219,
 14102220,
 14102221,
 14102222,
 14102223,
 14102300,
 14102301,
 14102302,
 14102303,
 14102304,
 14102305,
 14102306,
 14102307,
 14102308,
 14102309,
 14102310,
 14102311,
 14102312,
 14102313,
 14102314,
 14102315,
 14102316,
 14102317,
 14102318,
 14102319,
 14102320,
 14102321,
 14102322,
 14102323,
 14102400,
 14102401,
 14102402,
 14102403,
 14102404,
 14102405,
 14102406,
 14102407,
 14102408,
 14102409,
 14102410,
 14102411,
 14102412,
 14102413,
 14102414,
 14102415,
 14102416,
 14102417,
 14102418,

In [23]:
i=0
for data in datas:
    if i==0:
        for hour in hour_list:
            data_=data[data['hour']==hour]
            data_.to_csv('D:/Jupyter/w000/for_train/hour/'+str(hour),header=True,index_label=False)
    else:
        for hour in hour_list:
            data_=data[data['hour']==hour]
            data_.to_csv('D:/Jupyter/w000/for_train/hour/'+str(hour),header=False,mode='a',index_label=False)
    
    i+=1
    
#    if i>0:
#        break
    print(str(i*1000000))
    del data
    

1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000


In [24]:
aaa=pd.read_csv('D:/Jupyter/w000/for_train/hour/'+str(14102100))

In [25]:
aaa.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C16,C17,C18,C19,C20,C21,user_id,hour_hours,hour_weekday,hour_days
0,1.000009e+18,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,0,2,a99f214a44956a24,0,2,0
1,1.000017e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a711ee120,0,2,0
2,1.000037e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a8a4875bd,0,2,0
3,1.000064e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a6332421a,0,2,0
4,1.000068e+19,0,14102100,0,1,999,999,999,0,0,...,0,113,0,0,0,6,a99f214a779d90c2,0,2,0


#### 2.3.2 分别对生成的240小时的数据进行处理，将other编号为999的改为原特征数字最大表示+1.

In [36]:
aaa=read_double_dict('D:/FE_19.txt',key_int=False)
#生成每个特征中other特征的编号：为所有特征维数-1
length_dict={}
for k,v in aaa.items():
    name=[]
    for key,value in v.items():
        name.append(value)
    length=len(set(name))
    length_dict[k]=int(length)-1
length_dict

{'C1': 3,
 'C14': 372,
 'C15': 2,
 'C16': 3,
 'C17': 237,
 'C18': 3,
 'C19': 42,
 'C20': 75,
 'C21': 40,
 'app_category': 5,
 'app_domain': 15,
 'app_id': 144,
 'banner_pos': 2,
 'device_conn_type': 3,
 'device_model': 140,
 'device_type': 3,
 'site_category': 6,
 'site_domain': 109,
 'site_id': 128}

In [113]:
#生成240个小时的列表
hour_list=[]
for data in range(0,9):
    for hour in range(0,24):
        hour_list.append(14102100+data*100+hour)
hour_list

[14102100,
 14102101,
 14102102,
 14102103,
 14102104,
 14102105,
 14102106,
 14102107,
 14102108,
 14102109,
 14102110,
 14102111,
 14102112,
 14102113,
 14102114,
 14102115,
 14102116,
 14102117,
 14102118,
 14102119,
 14102120,
 14102121,
 14102122,
 14102123,
 14102200,
 14102201,
 14102202,
 14102203,
 14102204,
 14102205,
 14102206,
 14102207,
 14102208,
 14102209,
 14102210,
 14102211,
 14102212,
 14102213,
 14102214,
 14102215,
 14102216,
 14102217,
 14102218,
 14102219,
 14102220,
 14102221,
 14102222,
 14102223,
 14102300,
 14102301,
 14102302,
 14102303,
 14102304,
 14102305,
 14102306,
 14102307,
 14102308,
 14102309,
 14102310,
 14102311,
 14102312,
 14102313,
 14102314,
 14102315,
 14102316,
 14102317,
 14102318,
 14102319,
 14102320,
 14102321,
 14102322,
 14102323,
 14102400,
 14102401,
 14102402,
 14102403,
 14102404,
 14102405,
 14102406,
 14102407,
 14102408,
 14102409,
 14102410,
 14102411,
 14102412,
 14102413,
 14102414,
 14102415,
 14102416,
 14102417,
 14102418,

In [101]:
data=pd.read_csv(file_dir+str(14102100),nrows=5)
i=0
for key in length_dict.keys():
    data.loc[data[key]==999,key]=length_dict[key]
    i+=1

data.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C16,C17,C18,C19,C20,C21,user_id,hour_hours,hour_weekday,hour_days
0,1.000009e+18,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,0,2,a99f214a44956a24,0,2,0
1,1.000017e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a711ee120,0,2,0
2,1.000037e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a8a4875bd,0,2,0
3,1.000064e+19,0,14102100,0,0,1,1,2,0,0,...,0,0,0,0,1,2,a99f214a6332421a,0,2,0
4,1.000068e+19,0,14102100,0,1,128,109,6,0,0,...,0,113,0,0,0,6,a99f214a779d90c2,0,2,0


In [109]:
#将训练数据中的other由999替换成length_dict中的数字（除原始特征为的下一个数字）
def change_other(row,length_dict):
    for key in length_dict.keys():
        row.loc[row[key]==999,key]=length_dict[key]
    return row

file_dir='D:/Jupyter/w000/for_train/hour/'
save_dir='D:/Jupyter/w000/for_train/hour_new/'
i=0
for hour in hour_list:
    data=pd.read_csv(file_dir+str(hour))
    data=change_other(data,length_dict)
    print(str(i),str(hour))
    data.to_csv(save_dir+str(hour))
    del data
    
    i+=1    
    print(str(i),str(hour))
#    if i>0:
#        break

0 14102100
1 14102100
1 14102101
2 14102101
2 14102102
3 14102102
3 14102103
4 14102103
4 14102104
5 14102104
5 14102105
6 14102105
6 14102106
7 14102106
7 14102107
8 14102107
8 14102108
9 14102108
9 14102109
10 14102109
10 14102110
11 14102110
11 14102111
12 14102111
12 14102112
13 14102112
13 14102113
14 14102113
14 14102114
15 14102114
15 14102115
16 14102115
16 14102116
17 14102116
17 14102117
18 14102117
18 14102118
19 14102118
19 14102119
20 14102119
20 14102120
21 14102120
21 14102121
22 14102121
22 14102122
23 14102122
23 14102123
24 14102123
24 14102200
25 14102200
25 14102201
26 14102201
26 14102202
27 14102202
27 14102203
28 14102203
28 14102204
29 14102204
29 14102205
30 14102205
30 14102206
31 14102206
31 14102207
32 14102207
32 14102208
33 14102208
33 14102209
34 14102209
34 14102210
35 14102210
35 14102211
36 14102211
36 14102212
37 14102212
37 14102213
38 14102213
38 14102214
39 14102214
39 14102215
40 14102215
40 14102216
41 14102216
41 14102217
42 14102217
42 14102218

In [110]:
#将测试数据中other由999替换成length_dict中的数字（除原始特征为的下一个数字）
def change_other(row,length_dict):
    for key in length_dict.keys():
        row.loc[row[key]==999,key]=length_dict[key]
    return row

file_dir='D:/Jupyter/w000/for_train/'
save_dir='D:/Jupyter/w000/for_train/test_new/'

data=pd.read_csv(file_dir+'FE_test_26.csv')
data=change_other(data,length_dict)
data.to_csv(save_dir+'FE_test_26.csv')
del data



#### 2.3.3对处理好的特征进行下采样，形成不同大小的训练集、验证集。

In [30]:
def data_hour_combine(data_n=4,data_size=1000000,hour_list=hour_list,file_dir='D:/Jupyter/w000/for_train/hour_new/',save_dir='D:/Jupyter/w000/for_train/data_final/'):    
    i=0
    for hour in hour_list:
        data_num=data_size//216
        datas=pd.read_csv(file_dir+str(hour),chunksize=data_num)

        j=0
        for data in datas:
#由于频繁OI输入，输出导致permission denied，故而选择延时
            time.sleep(0.3)
            if i==0:
                data.to_csv(save_dir+str(data_size/10000)+'_'+str(j),header=True,index_label=False)
                del data
            else:
                data.to_csv(save_dir+str(data_size/10000)+'_'+str(j),header=False,mode='a',index_label=False) 
                del data
            j+=1
            if j==data_n:
                break
            
    
        i+=1  
#        if i>1:
#            break
        print(str(i),str(hour))

In [31]:
#生成4个100w大小的数据块
file_dir='D:/Jupyter/w000/for_train/hour_new/'
save_dir='D:/Jupyter/w000/for_train/data_final/'
data_hour_combine(4,1000000,hour_list,file_dir,save_dir)

1 14102100
2 14102101
3 14102102
4 14102103
5 14102104
6 14102105
7 14102106
8 14102107
9 14102108
10 14102109
11 14102110
12 14102111
13 14102112
14 14102113
15 14102114
16 14102115
17 14102116
18 14102117
19 14102118
20 14102119
21 14102120
22 14102121
23 14102122
24 14102123
25 14102200
26 14102201
27 14102202
28 14102203
29 14102204
30 14102205
31 14102206
32 14102207
33 14102208
34 14102209
35 14102210
36 14102211
37 14102212
38 14102213
39 14102214
40 14102215
41 14102216
42 14102217
43 14102218
44 14102219
45 14102220
46 14102221
47 14102222
48 14102223
49 14102300
50 14102301
51 14102302
52 14102303
53 14102304
54 14102305
55 14102306
56 14102307
57 14102308
58 14102309
59 14102310
60 14102311
61 14102312
62 14102313
63 14102314
64 14102315
65 14102316
66 14102317
67 14102318
68 14102319
69 14102320
70 14102321
71 14102322
72 14102323
73 14102400
74 14102401
75 14102402
76 14102403
77 14102404
78 14102405
79 14102406
80 14102407
81 14102408
82 14102409
83 14102410
84 14102411
8

In [32]:
#生成4个400w大小的数据块
file_dir='D:/Jupyter/w000/for_train/hour_new/'
save_dir='D:/Jupyter/w000/for_train/data_final/'
data_hour_combine(4,4000000,hour_list,file_dir,save_dir)

1 14102100
2 14102101
3 14102102
4 14102103
5 14102104
6 14102105
7 14102106
8 14102107
9 14102108
10 14102109
11 14102110
12 14102111
13 14102112
14 14102113
15 14102114
16 14102115
17 14102116
18 14102117
19 14102118
20 14102119
21 14102120
22 14102121
23 14102122
24 14102123
25 14102200
26 14102201
27 14102202
28 14102203
29 14102204
30 14102205
31 14102206
32 14102207
33 14102208
34 14102209
35 14102210
36 14102211
37 14102212
38 14102213
39 14102214
40 14102215
41 14102216
42 14102217
43 14102218
44 14102219
45 14102220
46 14102221
47 14102222
48 14102223
49 14102300
50 14102301
51 14102302
52 14102303
53 14102304
54 14102305
55 14102306
56 14102307
57 14102308
58 14102309
59 14102310
60 14102311
61 14102312
62 14102313
63 14102314
64 14102315
65 14102316
66 14102317
67 14102318
68 14102319
69 14102320
70 14102321
71 14102322
72 14102323
73 14102400
74 14102401
75 14102402
76 14102403
77 14102404
78 14102405
79 14102406
80 14102407
81 14102408
82 14102409
83 14102410
84 14102411
8

In [27]:
#生成4个800w大小的数据块
file_dir='D:/Jupyter/w000/for_train/hour_new/'
save_dir='D:/Jupyter/w000/for_train/data_final/'
data_hour_combine(4,8000000,hour_list,file_dir,save_dir)

1 14102100
2 14102101
3 14102102
4 14102103
5 14102104
6 14102105
7 14102106
8 14102107
9 14102108
10 14102109
11 14102110
12 14102111
13 14102112
14 14102113
15 14102114
16 14102115
17 14102116
18 14102117
19 14102118
20 14102119
21 14102120
22 14102121
23 14102122
24 14102123
25 14102200
26 14102201
27 14102202
28 14102203
29 14102204
30 14102205
31 14102206
32 14102207
33 14102208
34 14102209
35 14102210
36 14102211
37 14102212
38 14102213
39 14102214
40 14102215
41 14102216
42 14102217
43 14102218
44 14102219
45 14102220
46 14102221
47 14102222
48 14102223
49 14102300
50 14102301
51 14102302
52 14102303
53 14102304
54 14102305
55 14102306
56 14102307
57 14102308
58 14102309
59 14102310
60 14102311
61 14102312
62 14102313
63 14102314
64 14102315
65 14102316
66 14102317
67 14102318
68 14102319
69 14102320
70 14102321
71 14102322
72 14102323
73 14102400
74 14102401
75 14102402
76 14102403
77 14102404
78 14102405
79 14102406
80 14102407
81 14102408
82 14102409
83 14102410
84 14102411
8

In [33]:
#生成2个1200w大小的数据块
file_dir='D:/Jupyter/w000/for_train/hour_new/'
save_dir='D:/Jupyter/w000/for_train/data_final/'
data_hour_combine(4,12000000,hour_list,file_dir,save_dir)

1 14102100
2 14102101
3 14102102
4 14102103
5 14102104
6 14102105
7 14102106
8 14102107
9 14102108
10 14102109
11 14102110
12 14102111
13 14102112
14 14102113
15 14102114
16 14102115
17 14102116
18 14102117
19 14102118
20 14102119
21 14102120
22 14102121
23 14102122
24 14102123
25 14102200
26 14102201
27 14102202
28 14102203
29 14102204
30 14102205
31 14102206
32 14102207
33 14102208
34 14102209
35 14102210
36 14102211
37 14102212
38 14102213
39 14102214
40 14102215
41 14102216
42 14102217
43 14102218
44 14102219
45 14102220
46 14102221
47 14102222
48 14102223
49 14102300
50 14102301
51 14102302
52 14102303
53 14102304
54 14102305
55 14102306
56 14102307
57 14102308
58 14102309
59 14102310
60 14102311
61 14102312
62 14102313
63 14102314
64 14102315
65 14102316
66 14102317
67 14102318
68 14102319
69 14102320
70 14102321
71 14102322
72 14102323
73 14102400
74 14102401
75 14102402
76 14102403
77 14102404
78 14102405
79 14102406
80 14102407
81 14102408
82 14102409
83 14102410
84 14102411
8

#### 2.3.4 对每个小时数据进行特征工程获取每小时用户热度

#### $$$  获取每个小时中，每个用户观看广告C14、广告广告类别C17、登录指定app的统计表格

In [5]:
#1、根据user_id,hour,C14生成用户在每个小时段，登陆C14的次数表格
user_hour_C14_dict=data.groupby(['user_id', 'hour','C14'],as_index=False,squeeze=True).size().reset_index()
#2、根据user_id,hour,C17生成用户在每个小时段，登陆C17的次数表格
user_hour_C17_dict=data.groupby(['user_id', 'hour','C17'],as_index=False,squeeze=True).size().reset_index()
#6、根据user_id,hour_days，app_id生成用户每个小时登陆特定app的次数
user_hour_app_id_dict=data.groupby(['user_id', 'hour','app_id'],as_index=False,squeeze=True).size().reset_index()

AttributeError: 'int' object has no attribute 'groupby'

#### $$$  生成每个小时中，每个用户观看广告C14、广告广告类别C17、登录指定app的统计特征

In [2]:
#生成一个字典，用于存储10天的小时
hour_dict={}
for day in range(0,10):
    hour_l=[]
    for hour in range(0,24):
        hour_l.append(14102100+data*100+hour)
    hour_dict[day]=hour_l
hour_dict

{0: [14102900,
  14102901,
  14102902,
  14102903,
  14102904,
  14102905,
  14102906,
  14102907,
  14102908,
  14102909,
  14102910,
  14102911,
  14102912,
  14102913,
  14102914,
  14102915,
  14102916,
  14102917,
  14102918,
  14102919,
  14102920,
  14102921,
  14102922,
  14102923],
 1: [14102900,
  14102901,
  14102902,
  14102903,
  14102904,
  14102905,
  14102906,
  14102907,
  14102908,
  14102909,
  14102910,
  14102911,
  14102912,
  14102913,
  14102914,
  14102915,
  14102916,
  14102917,
  14102918,
  14102919,
  14102920,
  14102921,
  14102922,
  14102923],
 2: [14102900,
  14102901,
  14102902,
  14102903,
  14102904,
  14102905,
  14102906,
  14102907,
  14102908,
  14102909,
  14102910,
  14102911,
  14102912,
  14102913,
  14102914,
  14102915,
  14102916,
  14102917,
  14102918,
  14102919,
  14102920,
  14102921,
  14102922,
  14102923],
 3: [14102900,
  14102901,
  14102902,
  14102903,
  14102904,
  14102905,
  14102906,
  14102907,
  14102908,
  14102909,
 

##### 此为第1天的小时热度，由于并能实现并行，故而分别开了5个核分两次对特征进行抓取

In [16]:
#第一天数据新特征生成
file_dir='D:/Jupyter/w000/for_train/hour_26_new/'
save_dir='D:/Jupyter/w000/for_train/hour_29_new/'
i=0
for hour in hour_dict[0]:
    start=time.time()
    data=pd.read_csv(file_dir+str(hour))
    
#1、根据user_id,hour,C14生成用户在每个小时段，登陆C14的次数表格
    user_hour_C14_dict=data.groupby(['user_id', 'hour','C14'],as_index=False,squeeze=True).size().reset_index()
    user_hour_C17_dict=data.groupby(['user_id', 'hour','C17'],as_index=False,squeeze=True).size().reset_index()
    user_hour_app_id_dict=data.groupby(['user_id', 'hour','app_id'],as_index=False,squeeze=True).size().reset_index()    
        
    data['user_hour_C14'] = data.apply(lambda row: find_user_hour_C14(row['user_id'], row['hour'], row['C14'],user_hour_C14_dict), axis=1)
    data['user_hour_C17'] = data.apply(lambda row: find_user_hour_C17(row['user_id'], row['hour'], row['C17'],user_hour_C17_dict), axis=1)
    data['user_hour_app_id'] = data.apply(lambda row: find_user_hour_app_id(row['user_id'], row['hour'], row['app_id'],user_hour_app_id_dict), axis=1)
    
    data.to_csv(save_dir+str(hour))
    
    end=time.time()     
    print(i,data.shape,str(end-start))   
    
    i+=1    
#    if i>0:
#        break
    
    

0 (98950, 30) 892.8037421703339
1 (118914, 30) 1636.7186131477356
2 (141174, 30) 2327.893130540848
3 (228057, 30) 5936.765898227692
4 (160923, 30) 3311.5496287345886
5 (131740, 30) 2110.609743118286
6 (142879, 30) 2351.4976074695587
7 (228905, 30) 5886.655891895294
8 (201567, 30) 5251.328466653824
9 (186264, 30) 3737.3646183013916
10 (224107, 30) 4374.8272449970245
11 (161531, 30) 2932.538059234619
12 (192389, 30) 4146.622271776199
13 (193327, 30) 4449.445274591446
14 (224237, 30) 5576.896236181259
15 (202404, 30) 4321.818225622177
16 (177356, 30) 3592.220525741577
17 (158732, 30) 2961.136948108673
18 (132239, 30) 2421.838632106781
19 (125421, 30) 2221.493161678314
20 (117017, 30) 2003.7391300201416
21 (95752, 30) 1370.8487057685852
22 (90365, 30) 1159.7836031913757
23 (98358, 30) 1212.1846671104431


In [17]:
aaa=pd.read_csv(save_dir+str(14102900))
aaa.head()

,Unnamed: 0,Unnamed: 0.1,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,...,C19,C20,C21,user_id,hour_hours,hour_weekday,hour_days,user_hour_C14,user_hour_C17,user_hour_app_id
0,0,32377421,1.000007e+19,0,14102900,0,1,6,6,1,...,0,0,17,a99f214abe1b1994,0,3,8,11,30,36
1,1,32377422,1.000029e+19,0,14102900,0,0,1,1,2,...,0,7,11,a99f214ac503c652,0,3,8,13,13,18
2,2,32377423,1.000036e+19,0,14102900,0,1,39,38,1,...,10,12,3,a99f214a0710fff9,0,3,8,8,15,15
3,3,32377424,1.000042e+18,0,14102900,0,0,21,20,1,...,7,0,13,a99f214a1aa0e912,0,3,8,44,88,420
4,4,32377425,1.000049e+19,1,14102900,0,0,1,1,2,...,2,0,1,a99f214a711ee120,0,3,8,114,895,1601


#### 2.3.5 获取每天中反应用户热度特征

#### $$$  获取每天中反应用户热度的特征

In [24]:
#3、根据user_id,hour_days,C14生成用户在每天，登陆C14的次数表格
user_day_C14_dict=data.groupby(['user_id', 'hour_days','C14'],as_index=False,squeeze=True).size().reset_index()
#4、根据user_id,hour_days,C17生成用户在每天，登陆C14的次数表格
user_day_C17_dict=data.groupby(['user_id', 'hour_days','C17'],as_index=False,squeeze=True).size().reset_index()

#5、根据user_id,hour_days生成用户在每天出现次数表格
user_day_times_dict=data.groupby(['user_id', 'hour_days'],as_index=False,squeeze=True).size().reset_index()

#6、根据user_id,hour_days，app_id生成用户每天登陆特定app的次数
user_day_app_id_dict=data.groupby(['user_id', 'hour_days','app_id'],as_index=False,squeeze=True).size().reset_index()

#7、在user_day_times_dict的基础上对user_id进行排序，得到每个用户出现的天数
user_days_dict=user_day_times_dict.groupby(['user_id'],as_index=False,squeeze=True).size().reset_index()

##### 生成7个新特征数据
##### tip：10w数据差不多消耗20min

In [25]:
#相关find_user_hour_C14函数在FE_utils
start=time.time()
data['user_hour_C14'] = data.apply(lambda row: find_user_hour_C14(row['user_id'], row['hour'], row['C14'],user_hour_C14_dict), axis=1)
data['user_hour_C17'] = data.apply(lambda row: find_user_hour_C17(row['user_id'], row['hour'], row['C17'],user_hour_C17_dict), axis=1)
data['user_day_C14'] = data.apply(lambda row: find_user_day_C14(row['user_id'], row['hour_days'], row['C14'],user_day_C14_dict), axis=1)
data['user_day_C17'] = data.apply(lambda row: find_user_day_C17(row['user_id'], row['hour_days'], row['C17'],user_day_C17_dict), axis=1)

data['user_day_times'] = data.apply(lambda row: find_user_day_times(row['user_id'], row['hour_days'],user_day_times_dict), axis=1)
data['user_day_app_id'] = data.apply(lambda row: find_user_day_app_id(row['user_id'], row['hour_days'], row['app_id'],user_day_app_id_dict), axis=1)

data['user_days'] = data.apply(lambda row: find_user_days(row['user_id'],user_days_dict), axis=1)
end=time.time()
print(end-start)

6.26265287399292


In [26]:
#由于数据此前经历过下采样，正负样本相对集中，故而此处将数据的
data.sample(frac=1).reset_index(drop=True)
#将id、hour、user_id三个特征删除
data.drop(['id','hour','user_id'],inplace=True,axis=1)
#在删除了'id','hour','user_id'后，共有特征30个。
#其中类别型特征21个
#数值型特征共8个：hour_days、user_hour_C14、user_hour_C17、user_day_C14、user_day_C17、user_day_times、user_day_app_id、user_days
data.to_csv(save_name)